In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
# Read the file from the drive
file_path = '/content/drive/My Drive/FinalDataset.tsv'
df = pd.read_csv(file_path, delimiter='\t', error_bad_lines=False)

<ipython-input-4-c4991a1ac648>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, delimiter='\t', error_bad_lines=False)
Skipping line 17089: expected 15 fields, saw 22



In [ ]:
df_clean = df[['star_rating','review_body']]

In [ ]:
df_clean = df_clean.sample(frac=0.01)

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 15894 to 5907
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   star_rating  199 non-null    int64 
 1   review_body  199 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.7+ KB


In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import XLNetForSequenceClassification, XLNetTokenizer, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split

# Convert star ratings to sentiment classes
def convert_to_sentiment(rating):
    if rating in [1, 2]:
        return 0  # Negative
    elif rating == 3:
        return 1  # Neutral
    else:
        return 2  # Positive

df_clean['sentiment'] = df_clean['star_rating'].apply(lambda x: convert_to_sentiment(x))

In [ ]:
!pip install sentencepiece

In [ ]:
# Tokenization
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

def tokenize_reviews(review):
    tokens = tokenizer.encode_plus(review, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
    return tokens

df_clean['data'] = df_clean['review_body'].apply(lambda x: tokenize_reviews(x))

In [ ]:
import pandas as pd
import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Load your data (Assuming it's already loaded into df_clean)

# Convert the star ratings into sentiment values: 0 (negative), 1 (neutral), 2 (positive)
df_clean['label'] = pd.cut(df_clean['star_rating'], bins=[0,2,4,5], labels=[0, 1, 2], right=True).astype(int)

# Tokenization
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

def tokenize_reviews(review):
    tokens = tokenizer.encode_plus(review, max_length=512, truncation=True, padding='max_length', return_tensors='pt')
    return tokens

df_clean['data'] = df_clean['review_body'].apply(tokenize_reviews)

# Dataloader creation
def create_dataloader(df):
    input_ids = torch.stack(df['data'].apply(lambda x: x['input_ids']).tolist()).squeeze()
    attention_masks = torch.stack(df['data'].apply(lambda x: x['attention_mask']).tolist()).squeeze()
    labels = torch.tensor(df['label'].tolist(), dtype=torch.long)
    return DataLoader(TensorDataset(input_ids, attention_masks, labels), batch_size=8)

# Train/test split using train_test_split
train_df, val_df = train_test_split(df_clean, test_size=0.2)

train_loader = create_dataloader(train_df)
val_loader = create_dataloader(val_df)

# ... (The rest of your model training code goes here)


In [ ]:
# Model setup and training
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=3)
optimizer = AdamW(model.parameters(), lr=1e-5)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_masks, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_loader)
    print(f'Epoch: {epoch+1}, Training Loss: {avg_train_loss:.4f}')

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 1, Training Loss: 0.9919
Epoch: 2, Training Loss: 0.8858
Epoch: 3, Training Loss: 0.8596
Epoch: 4, Training Loss: 0.7714


In [ ]:
# Model evaluation
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_masks, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        predictions.extend(logits)
        true_labels.extend(label_ids)
    predicted_labels = [list(p).index(max(p)) for p in predictions]

In [ ]:
predicted_labels = [list(p).index(max(p)) for p in predictions]


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7250


In [ ]:
report = classification_report(true_labels, predicted_labels)
print(report)

from sklearn.metrics import confusion_matrix

# Compute and print the confusion matrix
print("Confusion Matrix:\n", confusion_matrix(true_labels, predicted_labels))


              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       0.33      0.11      0.17         9
           2       0.76      0.93      0.83        27

    accuracy                           0.73        40
   macro avg       0.61      0.60      0.58        40
weighted avg       0.66      0.72      0.68        40

Confusion Matrix:
 [[ 3  1  0]
 [ 0  1  8]
 [ 1  1 25]]
